In [6]:
import argparse
import io
import json
import os
from google.cloud import speech, storage

# DST_BUCKET_NAME = os.environ.get('DST_BUCKET_NAME') ---- Uncomment in Cloud Function

DST_BUCKET_NAME = "meeting-transcript-processed-team13"

In [7]:
def transcribe_gcs(gcs_uri: str) -> dict:
    """Asynchronously transcribes the audio file specified by the gcs_uri."""

    client = speech.SpeechClient()

    audio = speech.types.RecognitionAudio(uri=gcs_uri)
    
    config = speech.types.RecognitionConfig(
        encoding=speech.enums.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=48000,
        language_code="en-US",
        audio_channel_count=2,
        enable_automatic_punctuation=True,
    )

    operation = client.long_running_recognize(config=config, audio=audio)

    print("Waiting for operation to complete...")
    
    response = operation.result(timeout=2000)
   
    transcript = []
    for res in response.results:
        for alt in res.alternatives:
            transcript.append({"transcript": alt.transcript, "confidence": alt.confidence})
    
    return transcript

In [8]:
def upload_file_to_gcs(
        src_filename: str,
        transcript: dict,
        storage_client,
        DST_BUCKET_NAME: str,
):
    """Uploads a file to Google Cloud Storage."""

    dst_filename = os.path.join("PROCESSED_" + src_filename)
    tmp_filepath = os.path.join("/tmp", dst_filename)

    with open(f'{tmp_filepath}', 'w') as file:
        json.dump(transcript, file)

    print(f'Processed transcript stored in: {tmp_filepath}')

    bucket = storage_client.get_bucket(DST_BUCKET_NAME)
    blob = bucket.blob(dst_filename)

    blob.upload_from_filename(tmp_filepath)

    return print(f'{tmp_filepath} uploaded to {DST_BUCKET_NAME}/{dst_filename}.')


In [27]:
def process_transcripts(data, context = None):
    
    # Set up parameters
    FILE = data['name']
    SRC_BUCKET_NAME = data['bucket']
    
    print(f'Source Bucket: {SRC_BUCKET_NAME}')
    print(f'Source Filename: {FILE}')
    print(f'Destination Bucket: {DST_BUCKET_NAME}')   
    
    print('Starting Process Transcript...')
    
    # Instantiate a Google Cloud Storage client 
    storage_client = storage.Client()
    
    # Process transcript from speech to text 
    gcs_uri = "gs://meeting-audio-team13/test_rec.flac"
    transcript = transcribe_gcs(gcs_uri)
    

    # Upload data to gcs
#     upload_transcript_to_gcs(FILE, transcript, storage_client, DST_BUCKET_NAME)
    upload_file_to_gcs(FILE, transcript, storage_client, DST_BUCKET_NAME)
    
    print('Process Transcript Complete.')

In [28]:
gcs_uri = "gs://meeting-audio-team13/on_mute_flac.flac"
gcs_uri2 = "gs://meeting-audio-team13/test_rec.flac"

# transcribe_gcs(gcs_uri2)

In [29]:
if __name__ == "__main__":
    
    gcs_test_object = {
        "name": "test_rec.json",
        "bucket": "meeting-transcript-team13",
    }
    
    process_transcripts(gcs_test_object)

Source Bucket: meeting-transcript-team13
Source Filename: test_rec.json


NameError: name 'DST_BUCKET_NAME' is not defined

#### Want to remove deprecated version of google.cloud and use latest.

The submodules `enums` and `types` have been removed.

In [ ]:
!pip install google-cloud-speech==2.0.0

Removed any references to submodules `enums` and `types`. Ran successfully.

In [1]:
"""Final code in the corresponding Cloud Function."""

from google.cloud import speech, storage
import json
import os


def transcribe_gcs(gcs_uri: str) -> dict:
    """Asynchronously transcribes the audio file specified by the gcs_uri."""

    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)

    config = speech.RecognitionConfig(
#         encoding=speech.RecognitionConfig.AudioEncoding.FLAC,
        encoding=speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
        sample_rate_hertz=48000,
        language_code="en-GB",
        audio_channel_count=2,
        enable_automatic_punctuation=True,
    )

    operation = client.long_running_recognize(config=config, audio=audio)

    print("Waiting for operation to complete...")

    response = operation.result()

    transcript = []
    for res in response.results:
        for alt in res.alternatives:
            transcript.append(alt.transcript)

    return transcript

def upload_file_to_gcs(
        src_filename: str,
        transcript: list,
        storage_client,
        DST_BUCKET_NAME: str,
):
    """Uploads a file to Google Cloud Storage."""

#     dst_filename = os.path.join("TRANSCRIBED_" + src_filename.replace(".flac",".txt"))
    dst_filename = os.path.join("TRANSCRIBED_" + src_filename.replace(src_filename.split(".")[1], "txt"))
    tmp_filepath = os.path.join("/tmp", dst_filename)

    with open(f'{tmp_filepath}', 'w') as file:
        for item in transcript:
            file.write(f'{item}\n')

    print(f'Processed transcript stored in: {tmp_filepath}')

    bucket = storage_client.get_bucket(DST_BUCKET_NAME)
    blob = bucket.blob(dst_filename)

    blob.upload_from_filename(tmp_filepath)

    return print(f'{tmp_filepath} uploaded to {DST_BUCKET_NAME}/{dst_filename}.')

def process_transcripts(data, context = None):
    """Orchestrator function that runs everything sequentially."""

    # Set up parameters
    FILE = data['name']
    SRC_BUCKET_NAME = data['bucket']
    DST_BUCKET_NAME = os.environ.get('DST_BUCKET_NAME')
    
    print(f'Source Bucket: {SRC_BUCKET_NAME}')
    print(f'Source Filename: {FILE}')
    print(f'Destination Bucket: {DST_BUCKET_NAME}')

    print('Starting Process Transcript...')

    # Instantiate a Google Cloud Storage client 
    storage_client = storage.Client()

    # Process transcript from speech to text 
    gcs_uri = f"gs://{SRC_BUCKET_NAME}/{FILE}"
    transcript = transcribe_gcs(gcs_uri)

    # Upload data to gcs
    upload_file_to_gcs(FILE, transcript, storage_client, DST_BUCKET_NAME)

    print('Process Transcript Complete.')

In [2]:
if __name__ == "__main__":
    
    gcs_test_object = {
        "name": "3minzoommeeting.mp3",
        "bucket": "meeting-audio-team13",
    }
    
    # In the Cloud Function set this as an environment variable
    os.environ["DST_BUCKET_NAME"] = "meeting-transcript-team13"
    
    process_transcripts(gcs_test_object)

Source Bucket: meeting-audio-team13
Source Filename: 3minzoommeeting.mp3
Destination Bucket: meeting-transcript-processed-team13
Starting Process Transcript...
Waiting for operation to complete...
Processed transcript stored in: /tmp/TRANSCRIBED_3minzoommeeting.txt
/tmp/TRANSCRIBED_3minzoommeeting.txt uploaded to meeting-transcript-processed-team13/TRANSCRIBED_3minzoommeeting.txt.
Process Transcript Complete.


In [21]:
src_filename = "im_on_mute.m4a"

src_filename2 = src_filename.replace(src_filename.split(".")[1], "txt")

In [22]:
print("src_filename: ", src_filename)

print("src_filename2: ", src_filename2)

src_filename:  im_on_mute.m4a
src_filename2:  im_on_mute.txt
